In [4]:
# Natural Language Processing - Assignment 2
# Sentiment analysis for movie reviews

This notebook was created for you to answer question 2, 3, 4 and 5 from assignment 2. Please read the steps and the provided code carefully and make sure you understand them. You will be provided with a rough outline of functions, but you will need to fill most of them with your own code. 

The (red) comments at the beginning of each function explain what they should do, which parameters you should give as input and which variables should be returned by the function. After the blue comments "### student code here###' you should write your own code.

SyntaxError: invalid syntax (<ipython-input-4-f28961fcc18f>, line 4)

### Step 0 - Libraries
Make sure you have the needed libraries installed on your computer: Pandas, Numpy, NLTK...

### Step 1 - Load Data

In the first step, we are going to load the data in a Pandas DataFrame. Pandas DataFrames are a useful way of storing data. DataFrames are tables in which data can be accessed as columns, as rows or as individual cells. You can find more info on DataFrames here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

Read the code below and make sure you understand what is happening. Run the code to load your data.

In [6]:
import os
import re
import pandas as pd
import numpy as np
import glob

In [7]:
def get_path(filename):
    """
    Makes a list of all the paths that fit the search requirement
    
    :param filename: A regular expression that defines the search requirement for the filenames
    :return  Returns a list of all the pathnames
    """
    # place the movies folder in the same directory as this notebook
    current_directory = os.getcwd()

    # glob.glob() is a pattern-matching path finder, it searches for the reviews in the movies folder based on a Regular Expression
    paths = glob.glob(current_directory + '/movies/' + filename)
    
    if len(paths) == 0:
        print('Your file list is empty. The code looks for the folder '+current_directory+'/movies, but could not find it.')
    else: 
        print("You loaded: ", len(paths), "files")
    return paths

In [8]:
def load_data(pathset):
    """
    Loads the data into a dataframe
    
    :param pathset:  A list of paths
    :return  A dataframe with three columns: Path, Review (Text) and Label
    """
    # Files are named by sentiment (P for positive, N for negative)
    pattern = re.compile('P-train[0-9]*.txt')
    reviews = []
    labels = []
    df = pd.DataFrame(columns = ['Path', 'Review', 'Label'])
    for path in pathset:
        if re.search(pattern, path):
            text = open(path, "r").read()
            reviews.append(text)
            labels.append('Pos')
        else:
            text = open(path, "r").read()
            reviews.append(text)
            labels.append('Neg')
    df['Path'] = pathset
    df['Review'] = reviews
    df['Label'] = labels
    return df

In [9]:
paths = get_path('train/[NP]-train[0-9]*.txt')
data = load_data(paths)
data.head()

You loaded:  600 files


,Path,Review,Label
0,f:\master\courses\1a\natural language processi...,Once again Mr. Costner has dragged out a movie...,Neg
1,f:\master\courses\1a\natural language processi...,This is a pale imitation of 'Officer and a Gen...,Neg
2,f:\master\courses\1a\natural language processi...,"Years ago, when DARLING LILI played on TV, it ...",Neg
3,f:\master\courses\1a\natural language processi...,I was looking forward to this movie. Trustwort...,Neg
4,f:\master\courses\1a\natural language processi...,I gave this a 3 out of a possible 10 stars.\n\...,Neg


### Step 2 - Tokenization

In this step, you should write a tokenizer and compare it with an off-the-shelf one.

#### 2.1 Making your own tokenizer

In [58]:
def my_tokenizer(text):
    """
    The implementation of your own tokenizer
    
    :param text:  A string with a sentence (or paragraph, or document...)
    :return  A list of tokens
    """    
    ### student code here ###
    import re
    regular=re.compile("[a-zA-z']+|[0-9]+\.[0-9]+?|[^a-z0-9A-Z\s]+")
    tokenized_text=regular.findall(text)
    return tokenized_text

sample_string1 = "If you have the chance, watch it. Although, a warning, you'll cry your eyes out."
sample_string2 = "Night gathers, and now my watch begins. It shall not end until my death. I shall take no wife, hold no lands, father no children. I shall wear no crowns and win no glory. I shall live and die at my post. I am the sword in the darkness. I am the watcher on the walls. I am the fire that burns against the cold, the light that brings the dawn, the horn that wakes the sleepers, the shield that guards the realms of men. I pledge my life and honor to the Night’s Watch, for this night and all the nights to come." #Write two more sample sentences to tokenize 
sample_string3 = "Imagination is more important than knowledge"
print(my_tokenizer(sample_string1))
print(my_tokenizer(sample_string2))
print(my_tokenizer(sample_string3))

['If', 'you', 'have', 'the', 'chance', ',', 'watch', 'it', '.', 'Although', ',', 'a', 'warning', ',', "you'll", 'cry', 'your', 'eyes', 'out', '.']
['Night', 'gathers', ',', 'and', 'now', 'my', 'watch', 'begins', '.', 'It', 'shall', 'not', 'end', 'until', 'my', 'death', '.', 'I', 'shall', 'take', 'no', 'wife', ',', 'hold', 'no', 'lands', ',', 'father', 'no', 'children', '.', 'I', 'shall', 'wear', 'no', 'crowns', 'and', 'win', 'no', 'glory', '.', 'I', 'shall', 'live', 'and', 'die', 'at', 'my', 'post', '.', 'I', 'am', 'the', 'sword', 'in', 'the', 'darkness', '.', 'I', 'am', 'the', 'watcher', 'on', 'the', 'walls', '.', 'I', 'am', 'the', 'fire', 'that', 'burns', 'against', 'the', 'cold', ',', 'the', 'light', 'that', 'brings', 'the', 'dawn', ',', 'the', 'horn', 'that', 'wakes', 'the', 'sleepers', ',', 'the', 'shield', 'that', 'guards', 'the', 'realms', 'of', 'men', '.', 'I', 'pledge', 'my', 'life', 'and', 'honor', 'to', 'the', 'Night', '’', 's', 'Watch', ',', 'for', 'this', 'night', 'and', '

#### 2.2 Using an off-the-shelf tokenizer

In [57]:
#Now we are gonna compare the tokenizer you just wrote with the one from NLTK
#if you installed NLTK but never downloaded the 'punkt' tokenizer, uncomment the following lines:
#import nltk
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
import nltk
def nltk_tokenizer(text):
    """
    This function should apply the word_tokenize (punkt) tokenizer of nltk to the input text
    
    :param text:  A string with a sentence (or paragraph, or document...)
    :return  A list of tokens
    """     
    ### student code here ###    
    from nltk.tokenize import word_tokenize
    tokenized_text=word_tokenize(text)
    return tokenized_text

test_sentences = ["I like this assignment because:\n-\tit is fun;\n-\tit helps me practice my Python skills.",
        "I won a prize, but I won't be able to attend the ceremony.",
        "“The strange case of Dr. Jekyll and Mr. Hyde” is a famous book... but I haven't read it.",
        "I work for the C.I.A., and you?",
        "OMG #Twitter is sooooo coooool <3 :-) <-- lol...why do i write like this idk right? :) 🤷😂 🤖"]

for test_string in test_sentences:
    print(my_tokenizer(test_string))
    print(nltk_tokenizer(test_string))
    print("\n")
    

['I', 'like', 'this', 'assignment', 'because', ':', '-', 'it', 'is', 'fun', ';', '-', 'it', 'helps', 'me', 'practice', 'my', 'Python', 'skills.']
['I', 'like', 'this', 'assignment', 'because', ':', '-', 'it', 'is', 'fun', ';', '-', 'it', 'helps', 'me', 'practice', 'my', 'Python', 'skills', '.']


['I', 'won', 'a', 'prize', ',', 'but', 'I', "won't", 'be', 'able', 'to', 'attend', 'the', 'ceremony.']
['I', 'won', 'a', 'prize', ',', 'but', 'I', 'wo', "n't", 'be', 'able', 'to', 'attend', 'the', 'ceremony', '.']


['“', 'The', 'strange', 'case', 'of', 'Dr.', 'Jekyll', 'and', 'Mr.', 'Hyde', '”', 'is', 'a', 'famous', 'book.', '..', 'but', 'I', "haven't", 'read', 'it.']
['“', 'The', 'strange', 'case', 'of', 'Dr.', 'Jekyll', 'and', 'Mr.', 'Hyde', '”', 'is', 'a', 'famous', 'book', '...', 'but', 'I', 'have', "n't", 'read', 'it', '.']


['I', 'work', 'for', 'the', 'C.', 'I.', 'A.', ',', 'and', 'you', '?']
['I', 'work', 'for', 'the', 'C.I.A.', ',', 'and', 'you', '?']


['OMG', '#', 'Twitter', 'is', 

#### 2.3 Vocabulary
Now you need to tokenize the reviews data. Use the nltk_tokenizer() for splitting words. Apply heavy normalisation by removing punctuation (e.g. using string.punctuation) and transforming each sentence to lowercase. Also add a start ('&lt;S>') and end token ('&lt;E>') to each list of tokens. 
Your tokenized review should look something like this: \['&lt;S>', 'i', 'really', 'liked', 'the', 'movie', '&lt;E>'\]. Add a new column to the DataFrame containing the tokens of each review. 

In [ ]:
def tokenize_reviews(reviews):
    """
    This function should apply the nltk_tokenizer to each review in input
    
    :param text:  A list of reviews (strings)
    :return  A list of tokenized reviews
    """     
    tokenized_reviews = []
    ### student code here ###      
    
    
    return tokenized_reviews



In [ ]:
data['Toks'] = tokenize_reviews(data['Review'])
data.head()

Next, count the words/n-grams to get their frequencies. Then, answer the questions in the assignment PDF

Now build the vocabulary. Since many words occur only a few times, keep only the words that occur at least 25 times in your vocabulary. Remember: the vocabulary contains only the types and not the tokens.



In [ ]:
from nltk import ngrams
from itertools import chain

In [ ]:
# example on how to create n-grams based on the ngrams() function
text = "Hello there how do you like this function"
bigrams = list(ngrams(word_tokenize(text), 2))
bigrams

In [ ]:
# Make a single list of all your tokens for each n-gram 
# Subsequently give this list as input to the get_frequencies(ngrams) function

# We created the list for the unigrams for you using chains
# Find more info on chains here: https://docs.python.org/3/library/itertools.html#itertools.chain
unigrams = list(chain.from_iterable(data['Toks']))


# Now create the lists for bigrams and trigrams yourself
### Student code here ###


In [ ]:














def get_frequencies(ngrams):
    
    """
    Counts the times a word occurs
    
    :param ngrams:  A list of ngrams
    :return  A dictionary with the ngram as key and its count as value
    """
    
    ### Student code here ###

    ngram_frequencies = {}
    


    return ngram_frequencies

In [ ]:
unigram_frequencies = get_frequencies(unigrams)
bigram_frequencies = get_frequencies(bigrams)
trigram_frequencies = get_frequencies(trigrams)


### Answers the questions of step 2.2 here ###
# Tip: look into using lambda to sort a dictionary
    


In [ ]:
def get_vocabulary(unigram_frequ):
    
    """
    Creates a vocabulary by filtering out the words with a count less than 25
    
    :param unigram_frequ: Dictionary with tokens as key and count as value
    :return  The param dictionary without the words with a count less than 25
    """
    
    ### Student code here ###
    
    vocabulary = {}

        
    return vocabulary

In [ ]:
vocabulary = get_vocabulary(unigram_frequencies)
vocabulary

### Step 3 - Text classification with a unigram language model

In our dataset we have two classes: positive (Pos) and negative (Neg). For
each class, we will calculate a separate language model. This is the training
or learning phase. In the apply phase, we will classify new texts as positive or
negative. For testing our machine learning classifier, we apply the models on
the documents in the test part of the corpus.

#### 1. Training phase
* Build 2 language models for the training directory, one positive, one negative.
* How big is the positive/negative vocabulary?

In [ ]:
# Now, let's separate the training corpus to positive and negative, and build two vocabularies: a positive and a negative one

### Student answer here ###


#### 2. Testing phase
Now that we know the unigram counts for each class, as well as the size of our labelled vocabulary, we can classify our test data. 

1. Load your test data and write it into a DataFrame. Then tokenize the words and add them as a new column.
2. Classify your test data. To find the correct class for each point in the data set, implement the unigram formula for classification (see homework 2 assignment, question 3).
3. Write your resulting classification to a new file of the same format as the groundtruth file. 


In [ ]:
# First, read all the test data and tokenize it.
# You may use the functions defined at the beginning of the notebook. 

### Student code here ###


In [ ]:
# Second, classify each test file according to the formula 
# Hint: Don't forget to use the log probabilities, and how the calculation of P(w|c) changes when moving to log space!
# Use Laplace smoothing and define k

def classify_unigram(tokens):
    '''
    Classification function based on unigrams for one test file 
    
    :param tokens: List of tokens from one test file
    :returns classification label: 'P' for positive label and 'N' for negative label
    '''
    ### Student code here ###
    
    
    
    # return classification, write your own if-statement
    if # Probability of positive sentiment > probability of negative sentiment:
        return 'P'
    else:
        return 'N'

In [ ]:
# Classify test set
test_data['Preds_unigram'] = test_data['Toks'].apply(classify_unigram)

In [ ]:
# Write results file
filenames = test_data['Path'].apply(lambda x: (x.split('\\')[-1])[:-4]) #get a list of the filenames
results = open('results_unigram.txt', 'w')
for (filename, pred) in zip(filenames, test_data['Preds_unigram']):
    results.write(filename + ' ' + pred + '\n')
results.close()

### Step 4 - Text classification with a bigram language model

Now we will classify the same dataset again, but this time with bigram language models. The steps are similar, but think about how the language models will change.

#### 1. Training phase
* Build 2 language models for the training directory, one positive, one negative.


In [ ]:
# Start by separating the training corpus to positive and negative, and building the labelled vocabulary

### Student code here ###


#### 2. Testing phase
Now that we know the bigram counts for each label, as well as the size of our labelled vocabulary, we can label our test data similarly to before. 

To find the correct label, implement the bigram formula for classification and write your resulting classification to a new file. We can use the test_data already created in the previous exercise.


In [ ]:
# First, we need to process our test data to represent bigrams


In [ ]:
# Second, classify each test file according to the formula 
# Hint: Don't forget to use the log probabilities, and how the calculation of P(w|c) changes when moving to log space!
# And when using Laplace smoothing, also k can be defined

def classify_bigram(tokens):
    '''
    Classification function based on bigrams for one test file 
    
    :param tokens: List of bigram tokens from one test file
    :returns classification label: 'P' for positive label and 'N' for negative label
    '''
    ### Student code here ###
    
    
    # return classification, write your own if-statement
    if # Probability of positive sentiment > probability of negative sentiment:
        return 'P'
    else:
        return 'N'

In [ ]:
# Classify test set
test_data['Preds_bigram'] = test_data['Bigrams'].apply(classify_bigram)

In [ ]:
# Write results file
filenames = test_data['Path'].apply(lambda x: (x.split('\\')[-1])[:-4]) #get a list of the filenames
results = open('results_bigram.txt', 'w')
for (filename, pred) in zip(filenames, test_data['Preds_bigram']):
    results.write(filename + ' ' + pred + '\n')
results.close()

In [ ]:
test_data.head()

### Step 5 - Classifier performance
This last section computes the accuracy of your classifier(s). In your report, discuss the results and ways to improve the classifier. 

Additional bonus points can be awarded if you implement one suggestion (or more), test if it improves the performance, and discuss why (or why not).

In [ ]:
import sys

In [ ]:
def read_predictions(filename):
    """ 
    Read predictions into dictionary
    
    param filename: the name of the file you would like to read
    return a dictionary containing the filename and the label
    """
    d = {}
    with open(filename) as f:
        for line in f:
           (key, val) = line.split()
           d[key] = val
    return d

In [ ]:
def evaluate(result_file):
    '''
    Evaluates the performance of your model by calculating the accuracy
    
    param result_file: the name of the result file you would like to evaluate
    return the accuracy of the prediction
    '''

    # place the groundtruth file in the same directory as this notebook
    current_directory = os.getcwd()
    ground_truth_file = ''.join(glob.glob(current_directory + '/groundtruth.txt'))
    results_file = ''.join(glob.glob(current_directory + result_file))

    print( "Results file: ",  results_file )
    print( "Groundtruth file: ",  ground_truth_file )

    results_map = read_predictions(results_file)
    ground_truth_map = read_predictions(ground_truth_file)

    # Calculate accuracy and print incorrect predictions
    correct = 0
    for ID in ground_truth_map:
        label = ground_truth_map[ID]
        if ID not in results_map:
            print( "Missing predictions for " + ID)
        elif results_map[ID] == label:
            correct = correct + 1
        else:
            print( "Incorrect " + ID )
    
    accuracy = str(float(correct)/len(ground_truth_map))

    # Print summary
    print( str(correct) + " out of " + str(len(ground_truth_map)) + " were correct!")
    print( "accuracy " + accuracy)
    
    return accuracy

In [ ]:
unigram_performance = evaluate('/results_unigram.txt')
bigram_performance = evaluate('/results_bigram.txt')